In [1]:
%pip install -q -U "google-genai>=1.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 4.8 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [3]:
from google import genai

client = genai.Client() # the API is automatically loaded from the environement variable

MODEL_ID = "gemini-2.5-flash" # @param ["gemini-2.5-flash-lite","gemini-2.5-flash","gemini-2.5-pro","gemini-2.0-flash"] {"allow-input":true, isTemplate: true}

In [ ]:
#Chary Publications Pvt. Ltd.
#S.S. Forgings & Engineering Works

In [16]:
## Give Specific URL for Queries

from IPython.display import HTML, Markdown

response = client.models.generate_content(
    model=MODEL_ID,
    contents="""
    Check if "S.S. Forgings & Engineering Works" is a Supplier on B2B Ecommerce Platforms in India with Seller Profile Available.

    Check these Websites: tradeindia.com, justdial.com, indiamart.com & Look for Seller's Own Website.

    Give Response in Below Format:
    indiamart.com: Yes or No, URL: XXXXXXXXXXXXXXXXx
    justdial.com: Yes or No, URL: XXXXXXXXXXXXXXXXx
    tradeindia.com: Yes or No, URL: XXXXXXXXXXXXXXXXx
    Own Website: Yes or No, URL: XXXXXXXXXXXXXXXXX

    Guardrail:
    Only Say Yes if Correct Profile URL is available.
    """,
    config={"tools": [{"google_search": {}}]},
)

# print the response
display(Markdown(f"Response:\n {response.text}"))
# print the search details
print(f"Search Query: {response.candidates[0].grounding_metadata.web_search_queries}")
# urls used for grounding
print(f"Search Pages: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")

print(response.candidates[0].grounding_metadata)

display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Response:
 indiamart.com: Yes, URL: https://www.indiamart.com/s-s-forgings-engineering-works/
justdial.com: Yes, URL: https://www.justdial.com/Delhi/S-S-Forgings-Engg-Works-New-Delhi/011PXX11-XX11-160520112423-D1T4_BZDET
tradeindia.com: Yes, URL: https://www.tradeindia.com/s-s-forgings-engineering-works/
Own Website: No, URL: XXXXXXXXXXXXXXXXX

Search Query: ['S.S. Forgings & Engineering Works indiamart.com seller profile', 'S.S. Forgings & Engineering Works justdial.com seller profile', 'S.S. Forgings & Engineering Works tradeindia.com seller profile', 'S.S. Forgings & Engineering Works official website']
Search Pages: indiamart.com, justdial.com, tradeindia.com
google_maps_widget_context_token=None grounding_chunks=[GroundingChunk(
  web=GroundingChunkWeb(
    title='indiamart.com',
    uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQEyar4PxTSZ8prkPwHEHdvvUpt9snUiunqjZJVEnDrUMrgMaM93zT1cERAHgnYnX70H0ThZyk5RWYzjHrST2w15bLeThfIl-B6vgqYUxhoZomKUS11pGppJ0rzzdSlACxvkHz-Fi4Egl4-O0J2_ybCchLrJ4VULKxB2jBkB2I1P'
  )
), GroundingChunk(
  web=GroundingChunkWeb(
    title='justdial.com',
    uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQFb19osZ7VAipMu9JT48FFVAM1IxbYfn6dhTKJrO9_ByZAWqvLliPTsNnSu0WsRuaibNLnHWMYGfBc4ywkPrFJDPyqkpn6dF5wnSO6L-nue0mo5qFZt3neajHJMbYbwCrD-ws4RVvY6x2CwTrTU3To

In [17]:
data = response.candidates[0].grounding_metadata

import requests

def resolve_redirect(url):
    try:
        resp = requests.head(url, allow_redirects=True, timeout=10)
        # HEAD is faster since we just want the redirect target
        return resp.url
    except Exception as e:
        return f"Error: {e}"

def extract_and_resolve(data):
    results = {}
    chunks = data.grounding_chunks or []

    for support in data.grounding_supports or []:
        # domain/source name is always before the ":"
        source = support.segment.text.split(":")[0].strip()

        ref = support.grounding_chunk_indices
        uris = [chunks[i].web.uri for i in ref if i < len(chunks)]
        resolved = [resolve_redirect(u) for u in uris]

        results[source] = resolved

    return results

data = response.candidates[0].grounding_metadata
final_links = extract_and_resolve(data)

for source, links in final_links.items():
    print(f"{source}:")
    for link in links:
        print(" ->", link)

indiamart.com:
 -> https://www.indiamart.com/ssforgingsandengineeringworks/profile.html
justdial.com:
 -> https://www.justdial.com/Delhi/S-s-Forgings-Engg-Works-New-Delhi/011PXX11-XX11-131026150038-Y4H2_BZDET
tradeindia.com:
 -> https://www.tradeindia.com/s-s-forgings-engineering-works-103835918/


In [18]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig

client = genai.Client()
model_id = "gemini-2.5-flash"

tools = [
  {"url_context": {}},
]

url1 = "https://www.tradeindia.com/s-s-forgings-engineering-works-103835918/"

response = client.models.generate_content(
    model=model_id,
    contents=f"Describe in detail {url1}",
    config=GenerateContentConfig(
        tools=tools,
    )
)

for each in response.candidates[0].content.parts:
    print(each.text)

# For verification, you can inspect the metadata to see which URLs the model retrieved
print(response.candidates[0].url_context_metadata)

S.S. Forgings & Engineering Works, based in New Delhi, Delhi, India, is a company established in 1991, with Mr. Jabbar as its proprietor. The company is recognized as an exporter, manufacturer, supplier, and service provider, specializing in industrial screening and material handling equipment. They are a "Ti Trusted Seller" and "Premium Seller" on Tradeindia.com.

Their product range includes industrial screening and material handling equipment designed for high performance, catering to various industries such as mining, metallurgy, and recycling. Some of their diverse products are Rotary Screens, Rotary Trommel Screens, Vibrator Screening Machines, Scrap Rotary Screen Machines, Zinc Ash Crushing Machines, Ball Grinding Mill Machines, Magnetic Vibrator Belt Conveyors, Vibrator Belt Conveyors, and Aluminium Dross Ball Mill Machines.

The company emphasizes designing its products with the latest technology and robust materials to ensure efficiency, durability, and high performance. They

In [21]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig

client = genai.Client()
model_id = "gemini-2.5-flash"

tools = [
  {"url_context": {}},
]

url1 = "https://www.justdial.com/Delhi/SK-Tubewell-Engineers-And-Constuction-Nr-Jhole-Lal-Mandir-Moti-Nagar/"

response = client.models.generate_content(
    model=model_id,
    contents=f"Describe in detail {url1} & Check if Profile has Trust or Verified Flags",
    config=GenerateContentConfig(
        tools=tools,
    )
)

for each in response.candidates[0].content.parts:
    print(each.text)

# For verification, you can inspect the metadata to see which URLs the model retrieved
print(response.candidates[0].url_context_metadata)


The Justdial page for "SK Tubewell Engineers And Constuction" in Moti Nagar, Delhi, provides information about the business. The company has a rating of 4.4 out of 5 stars based on 48 ratings.

Regarding the "Trust" and "Verified" flags:
*   The profile explicitly displays an icon stating "This business Information is trust by Justdial."
*   It also displays an icon stating "This business Information is verified by Justdial."

The business operates from New Moti Nagar, Moti Nagar, Delhi, near Jhole Lal Mandir, with the postal code 110015. They offer various services including:
*   Mico Borewell Submersible Pump Repair And Services (₹2,000 - ₹3,000)
*   Submersible Pump Repair And Services (₹1,000 - ₹2,000)
*   Agriculture Borewell Drilling Services (₹1,200 - ₹1,500 / Foot)

The business has been in operation for 31 years and is listed under categories like Borewell Contractors, Submersible Pump Repair & Services, and Contractors. They are open from 9:30 am to 6:30 pm. They also provid